## Global Imports

In [1]:
import pandas
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
london = pandas.read_csv("/home/vivek/Documents/mydata/iimb/regression/regression_input_post_code_year_month_aggregate.csv")

/home/vivek/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (142) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
london.shape

(2189743, 152)

## Dummifying Month And Cluster Columns

In [2]:
london["month"] = london["year_month_new"].map(lambda x:int(str(x)[4:]))
monthDummies = pandas.get_dummies(london["month"],prefix="month"+"_")
london = pandas.concat([london,monthDummies],axis=1)
london.drop(["month"],axis=1,inplace=True)

clusterDummies = pandas.get_dummies(london["cluster"],prefix="cluster"+"_")
london = pandas.concat([london,clusterDummies],axis=1)
london.drop(["cluster"],axis=1,inplace=True)

In [3]:
londonPrice = london[london["year_month_new"]>=200901]  ## 199501 for full data
dropList = ["borough","ward","borough_code","latitude","longitude","yearBetween",\
            "YearMon","Year","Month","ward_code",'avg_co2_emission_in_kt', 'avg_population']
londonPrice = londonPrice.drop(dropList,axis=1)

train = londonPrice[londonPrice["year_month_new"]<201401]
test = londonPrice[londonPrice["year_month_new"]>=201401]
testCols = train.columns[3:]

## Gradient Boosting

In [14]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
gbm = GradientBoostingRegressor()
param_grid = { 
    'n_estimators': [3000],
    'max_features': ["sqrt"],
    'min_samples_leaf':[5],
    'min_samples_split':[5],
#     'max_depth' : [15],
    'subsample': [0.8],
    'learning_rate':[0.01],
    'criterion':["mse"],
    'verbose':[1]
}
grid_gbm = GridSearchCV(estimator=gbm, param_grid=param_grid, cv= 2)
X_train = train[testCols]
y = train["price"]
grid_gbm.fit(X_train,y)

# Make predictions
predsTrain =grid_gbm.predict(X= train[testCols])
predsTest = grid_gbm.predict(X= test[testCols])

      Iter       Train Loss      OOB Improve   Remaining Time 
         1 460414963222.7638  1273784234.9982            5.34m
         2 456097446626.0076  2433710850.0869            5.32m
         3 457936795388.9999  1557800783.4706            5.45m
         4 447452330129.0650  1303070696.7836            5.71m
         5 448044461274.0524  1723526408.5315            5.80m
         6 446093940704.8941  2599989996.2524            5.85m
         7 445896995372.6137  2023894056.6814            5.76m
         8 458094946294.9720  1321723778.7716            5.72m
         9 451081879401.4693  2056799323.2302            5.69m
        10 453678256383.8464  1082699920.0403            5.89m
        20 438692538455.5643  1460114491.4177            5.37m
        30 419359619252.9931  1230160538.5576            5.27m
        40 388549247143.2611  1082968635.5794            5.14m
        50 408094363296.7012  1355530732.3416            5.09m
        60 370637830674.5212  1931763012.4727          

## Calculating Train And Test MAPE

In [15]:
print (grid_gbm.best_params_)
trainResults = train
trainResults["Predicted"] = predsTrain
postalMeanTrain = pandas.DataFrame(trainResults.groupby(["post_code"])["price","Predicted"].mean()).reset_index()
postalMeanTrain["AP"] = abs(postalMeanTrain["price"]  -  postalMeanTrain["Predicted"])/postalMeanTrain["price"]
print("Train Mape: ",postalMeanTrain.AP.mean())

testResults = test
testResults["Predicted"] = predsTest
postalMeanTest = pandas.DataFrame(testResults.groupby(["post_code"])["price","Predicted"].mean()).reset_index()
postalMeanTest["AP"] = abs(postalMeanTest["price"]  -  postalMeanTest["Predicted"])/postalMeanTest["price"]
print("Test Mape: ",postalMeanTest.AP.mean())



{'criterion': 'mse', 'learning_rate': 0.01, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 3000, 'subsample': 0.8, 'verbose': 1}
Train Mape:  0.2658982737908239
Test Mape:  0.2769334513925649


## Running Gradient Boost For Individual Cluster

In [ ]:
for clusNum in list(london["cluster"].unique()):
    print("cluster Num: ",clusNum)
    londonPrice = london[london["cluster"]==int(clusNum)]

    londonPrice = londonPrice[londonPrice["year_month_new"]>=200901] #199501 for full data
    dropList = ["borough","ward","borough_code","latitude","longitude","yearBetween",\
                "YearMon","Year","Month","ward_code",'avg_co2_emission_in_kt', 'avg_population']
    londonPrice = londonPrice.drop(dropList,axis=1)

    train = londonPrice[londonPrice["year_month_new"]<201401]
    test = londonPrice[londonPrice["year_month_new"]>=201401]
    testCols = train.columns[3:]

    X_train = train[testCols]
    y = train["price"]

    # Initialize logistic regression model
    gbm = GradientBoostingRegressor(n_estimators=5000,learning_rate=0.01,subsample=0.8,\
                                    max_features="sqrt",min_samples_split=5)
    # Train the model
    gbm.fit(X = X_train,y = y)

    # Make predictions
    predsTrain =gbm.predict(X= train[testCols])
    predsTest = gbm.predict(X= test[testCols])


    trainResults = train
    trainResults["Predicted"] = predsTrain
    postalMeanTrain = pandas.DataFrame(trainResults.groupby(["post_code"])["price","Predicted"].mean()).reset_index()
    postalMeanTrain["AP"] = abs(postalMeanTrain["price"]  -  postalMeanTrain["Predicted"])/postalMeanTrain["price"]
    print("Train Mape: ",postalMeanTrain.AP.mean())

    testResults = test
    testResults["Predicted"] = predsTest
    postalMeanTest = pandas.DataFrame(testResults.groupby(["post_code"])["price","Predicted"].mean()).reset_index()
    postalMeanTest["AP"] = abs(postalMeanTest["price"]  -  postalMeanTest["Predicted"])/postalMeanTest["price"]
    print("Test Mape: ",postalMeanTest.AP.mean())

    print ("R2",gbm.score(X = X_train,y = y))
    print("######## End Of Iteration ######\n")